# Implementation of the SIMCLR with resnet50 backbone

In [1]:
from dotenv import load_dotenv
load_dotenv()       # reads .env and sets os.environ
import wandb
wandb.login()

wandb: Currently logged in as: analiju (analiju-paris) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
import os
import torch
import torch.nn as nn
from torchvision.models import resnet50
import numpy as np
from utils.version_utils import print_versions, configure_gpu_device, set_seed

In [ ]:
from simclr.data.datamodule import compute_mean_std, prepare_data, combine_train_val_loaders, SimCLRDataset, get_split_indexes, get_data_loaders


from simclr.data.transforms import TwoCropsTransform, get_transforms
    
from simclr.models.simclr import SimCLRModel, ProjectionHead


from simclr.models.loss import NTXentLoss, compute_contrastive_val_loss, compute_contrastive_accuracy


from simclr.data.datamodule import LabeledEvalDataset

from simclr.probes.logistic import get_probe_loaders, run_logistic_probe_experiment

from simclr.utils.scheduler import make_optimizer_scheduler

from simclr.utils.misc import evaluate

from simclr.data.mydataloaders import get_data_loaders_train_test_linear_probe
from simclr.config import CONFIG
from simclr.train import train_simclr


In [4]:

print_versions()
set_seed(seed=42)

TARGET_GPU_INDEX = CONFIG["TARGET_GPU_INDEX"] if "TARGET_GPU_INDEX" in CONFIG else 0  # Default to 0 if not set

DEVICE = configure_gpu_device(TARGET_GPU_INDEX)

Conda version: 25.5.1
Python version: 3.10.16
PyTorch version: 2.5.1
CUDA available: True
CUDA device count: 3
Torchvision version: 0.20.1
Successfully set to use GPU: 0 (Quadro RTX 6000)
Final DEVICE variable is set to: cuda:0
Current PyTorch default device: 0
Current PyTorch default device (after set_device): 0
Dummy tensor is on device: cuda:0


In [5]:

# Prevent nondeterminism
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
torch.backends.cudnn.enabled = False


DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# split fractions
TRAIN_FRAC = CONFIG["TRAIN_FRAC"]
VAL_FRAC   = CONFIG["VAL_FRAC"]
TEST_FRAC  = CONFIG["TEST_FRAC"]

SEED = CONFIG["SEED"]

PRETRAINED = False

TEMPERATURE = CONFIG["TEMPERATURE"]

BETAS=(0.9,0.98)
EPS = 1e-8

GLOBAL_SEED = CONFIG["SEED"]
NUM_WORKERS = CONFIG["NUM_WORKERS"]

EUROSAT_IMAGE_SIZE = (64, 64)
MODEL_INPUT_SIZE = [224, 224]
EPOCH_SAVE_INTERVAL = CONFIG["EPOCH_SAVE_INTERVAL"]

In [6]:

seeds = [GLOBAL_SEED]
for seed in seeds:
    print(f"\n=== Starting run with seed {seed} ===")
    set_seed(seed)
    
    data_dir = prepare_data()
    train_loader, val_loader, test_loader, val_subset_no_transform, num_classes = get_data_loaders(data_dir, CONFIG["BATCH_SIZE"])


=== Starting run with seed 42 ===
Total samples in folder: 27000, classes: ['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture', 'PermanentCrop', 'Residential', 'River', 'SeaLake']
Stratified split sizes: train=21600, val=2700, test=2700
Computed mean: [0.3441457152366638, 0.3800986111164093, 0.40766361355781555]
Computed std:  [0.09299743920564651, 0.06464490294456482, 0.054139167070388794]
Mean and std saved to models/mean_std.txt
Train/Val/Test loaders: 84/11/11 batches


In [ ]:

seeds = [GLOBAL_SEED]
for seed in seeds:
    print(f"\n=== Starting run with seed {seed} ===")
    set_seed(seed)
    
    data_dir = prepare_data()
    train_loader, val_loader, test_loader, val_subset_no_transform, num_classes = get_data_loaders(data_dir, CONFIG["BATCH_SIZE"])

    base_encoder = resnet50(weights=None)
    simclr_model = SimCLRModel(base_encoder, proj_dim=CONFIG["PROJ_DIM"])
    # optimizer = optim.Adam(simclr_model.parameters(), lr=CONFIG["LR"])
    wd =  0.5 
    optimizer, scheduler = make_optimizer_scheduler(
        simclr_model.parameters(),
        CONFIG["LR"],
        CONFIG["WD"],
        len(train_loader),
        CONFIG["EPOCHS_SIMCLR"]
        )
    
    bs = CONFIG["BATCH_SIZE"]
    loss_fn = NTXentLoss(bs, temperature=TEMPERATURE, device=DEVICE)

    print("Starting SimCLR training...")
    epochs_simclr = CONFIG["EPOCHS_SIMCLR"]
    lr = CONFIG["LR"]
    wandb_run = wandb.init(
        project="eurosat-contrastive-scratch",
        name=f"BS{bs}_LR{lr:.0e}_SEED{seed}_TEMPERATURE{TEMPERATURE}_EPOCHS{epochs_simclr}",
        tags=["SimCLR", "EuroSAT", "Contrastive Learning"],
        config={
            "seed": seed,
            "temperature": TEMPERATURE,
            "model": "SimCLR",
            "dataset": "EuroSAT",
            "batch_size": bs,
            "learning_rate": CONFIG["LR"],
            "epochs": CONFIG["EPOCHS_SIMCLR"],
            "proj_dim": CONFIG["PROJ_DIM"],
            "feature_dim": CONFIG["FEATURE_DIM"],
            "pretrained": PRETRAINED,
        }
    )

    eval_transform, augment_transform = get_transforms(
        mean =CONFIG["MEAN"],
        std = CONFIG["STD"]
    )  # these must match the transforms used in test_loader

    probe_train_loader, probe_val_loader = get_probe_loaders(
        train_loader,
        val_loader,
        eval_transform,               # must match transforms used in test_loader
        probe_batch_size=CONFIG["BATCH_SIZE"]
    )

    train_simclr(
        simclr_model,
        train_loader, val_loader,
        probe_train_loader, probe_val_loader,
        optimizer, loss_fn, DEVICE,
        simclr_epochs=CONFIG["EPOCHS_SIMCLR"],
        probe_lr=CONFIG["LR_LINEAR"],
        probe_epochs=1,            # 1 pass per epoch is typical
        feature_dim=CONFIG["FEATURE_DIM"],
        num_classes=num_classes,
        augment_transform=augment_transform,
        val_subset_no_transform=val_subset_no_transform,
        wandb_run=wandb_run,
        scheduler=scheduler,
        seed=seed
    )

    wandb_run.finish()



print("All runs completed.")
wandb.finish()
   


=== Starting run with seed 42 ===
Total samples in folder: 27000, classes: ['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture', 'PermanentCrop', 'Residential', 'River', 'SeaLake']
Stratified split sizes: train=21600, val=2700, test=2700
Computed mean: [0.3441457152366638, 0.3800986111164093, 0.40766361355781555]
Computed std:  [0.09299743920564651, 0.06464490294456482, 0.054139167070388794]
Mean and std saved to models/mean_std.txt
Train/Val/Test loaders: 84/11/11 batches
Starting SimCLR training...


/users/c/carvalhj/miniconda3/envs/myenv/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:240: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
/users/c/carvalhj/miniconda3/envs/myenv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1264: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/users/c/carvalhj/miniconda3/envs/myenv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:470: 

[probe] val accuracy = 54.07%


/users/c/carvalhj/miniconda3/envs/myenv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1264: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/users/c/carvalhj/miniconda3/envs/myenv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 200 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=200).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[probe] val accuracy = 78.63%
Epoch 01/2 | Train Loss: 6.1338, Val Loss: 6.0011 | Logistic Probe Acc (Val): 0.541, Logistic Probe Acc (Train): 0.786 | Contrastive Acc (Train): 0.008, Contrastive Acc (Val): 0.012 | KNN Acc (Val): 0.464


In [ ]:
# get the saved model and run linear probe
seed = CONFIG["SEED"]
bs = CONFIG["BATCH_SIZE"]
epochs_simclr = CONFIG["EPOCHS_SIMCLR"]
simclr_lr = CONFIG["LR"]
lr_str = f"{simclr_lr:.0e}" if simclr_lr < 0.0001 else f"{simclr_lr:.6f}"
model_path = f"models/simclr_seed{seed}_bs{bs}_temp{TEMPERATURE}_Tepochs{epochs_simclr}_lr{lr_str}.pth"

if not os.path.exists(model_path):
    print(f"Model {model_path} does not exist. Please run the SimCLR pretraining first.")

base_encoder = resnet50(weights=None)
simclr_model = SimCLRModel(base_encoder, proj_dim=CONFIG["PROJ_DIM"])
checkpoint_path = model_path
state_dict = torch.load(checkpoint_path, map_location=torch.device(DEVICE), weights_only=True)
simclr_model.load_state_dict(state_dict)

# Perform linear probe on train+val as train set, and test as test set
train_loader, test_loader, num_classes = get_data_loaders_train_test_linear_probe(CONFIG["DATA_DIR_LOCAL"], CONFIG["BATCH_SIZE"])
run_logistic_probe_experiment(
    42,
    train_loader,
    None,  # No validation loader for linear probe
    test_loader,
    num_classes,
    simclr_model,
    bs
)


In [ ]:
# grid search for best hyperparameters

batch_sizes_epochs = [
    (64, 35),
    (128, 40),
    (256, 100),
    (512, 100),
    (1024, 150),
]

learning_rates = [
    1e-3,
    3.75e-4,
    1e-4,
    3.75e-5,
    1e-5,
]

# use linspace for computing the temperature
temperatures = np.linspace(0.05, 0.5, 5).tolist() # [0.05, 0.1625, 0.275, 0.3875, 0.5]
temperatures.append(0.2)  # add the original temperature

gpu_indexes = [0, 1]
# put half of the experiments on each GPU
gpu_experiments = {0: [], 1: []}
all_acc = []

# train simclr with different hyperparameters and apply linear probe